<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_SSE/test_sample_SSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

Cloning into 'PyTorch-Architectures'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 1066 (delta 59), reused 88 (delta 32), pack-reused 938
Receiving objects: 100% (1066/1066), 8.48 MiB | 20.10 MiB/s, done.
Resolving deltas: 100% (630/630), done.


In [2]:
%cd PyTorch-Architectures/modeling_SSE/

/content/PyTorch-Architectures/modeling_SSE


In [3]:
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe
import torch
import torch.nn as nn
from model import BiLSTMSE

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
tokenize = lambda x: x.split()
TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True, batch_first=True, fix_length=200)
LABEL = data.LabelField()
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))
LABEL.build_vocab(train_data)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:05<00:00, 16.5MB/s]
.vector_cache/glove.6B.zip: 862MB [06:27, 2.22MB/s]                           
100%|█████████▉| 399245/400000 [00:45<00:00, 8397.48it/s]

In [5]:
word_embeddings = TEXT.vocab.vectors

In [6]:
train_data, valid_data = train_data.split()
train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=32, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)

In [13]:
# Parameters
batch_size = 32
vocab_size = len(TEXT.vocab)
emb_dim = 300
hidden_dim = 300
n_layers = 2
natt_unit = 300
natt_hops = 1
nfc = 512
n_class = 2
drop_prob = 0.5
learning_rate = 0.001

In [11]:
model = BiLSTMSE(
    batch_size=batch_size,
    vocab_size=vocab_size,
    emb_dim=emb_dim,
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    natt_unit=natt_unit,
    natt_hops=natt_hops,
    nfc=nfc,
    n_class=n_class,
    drop_prob=drop_prob,
    weights=word_embeddings,
)
model = model.to(device)

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
EPOCHS = 4
for epoch in range(EPOCHS):
  model.train()
  for idx, batch in enumerate(train_iter):
    text = batch.text[0].to(device)
    label = batch.label.to(device)

    if text.size(0) is not 32: # irregular behaviour in one batch
      continue
    
    model = 